# Intro to Transfer Learning

## Import Packages

In [1]:
import pandas as pd
import numpy as np

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout
from sklearn.preprocessing import StandardScaler


## Prepare the Data

In [3]:
df = pd.read_csv('../Resources/meet_or_beat_US.csv')
df

,EPS,forecasted_eps,noOfEsts,after_total_returns,before_total_returns
0,2.01,1.67,11,0.051444,0.018585
1,0.17,0.19,6,0.112955,-0.000510
2,-0.07,0.14,4,0.077167,-0.046104
3,0.48,0.51,8,-0.006130,-0.004899
4,-0.24,-0.27,9,0.089762,-0.025466
...,...,...,...,...,...
16834,2.16,0.22,1,-0.011062,-0.041509
16835,-0.16,-0.20,3,0.322034,-0.040698
16836,-0.16,-0.15,7,0.101928,0.073204
16837,-2.58,-3.45,1,0.099432,0.091058


In [4]:
X = df[['EPS','forecasted_eps','noOfEsts','before_total_returns']]
y = df[['after_total_returns']]

In [5]:
# Split into training and testing windows
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [6]:
# Create the StandardScaler instance
X_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Scale both the training and testing data from the features dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Build a Model

In [7]:
# Define the the number of inputs to the model
number_inputs = 4

# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 64

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 32

# Create the Sequential model instance
nn = Sequential()

# Add the first Dense layer specifying the number of inputs, the number of hidden nodes, and the activation function
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_inputs, activation="relu"))

# Add the second Dense layer specifying the number of hidden nodes and the activation function
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

nn.add(Dropout(.2,input_shape=(hidden_nodes_layer2,)))

# Add the second Dense layer specifying the number of hidden nodes and the activation function
nn.add(Dense(units=24, activation="relu"))

nn.add(Dropout(.2,input_shape=(24,)))

# Add the second Dense layer specifying the number of hidden nodes and the activation function
nn.add(Dense(units=12, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(1))

Metal device set to: Apple M1 Max


2022-03-15 21:09:21.558214: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-15 21:09:21.558325: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
# Summarise the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                320       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 24)                792       
                                                                 
 dropout_1 (Dropout)         (None, 24)                0         
                                                                 
 dense_3 (Dense)             (None, 12)                300       
                                                                 
 dense_4 (Dense)             (None, 1)                 1

In [9]:
# Compile the Sequential model
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

In [10]:
# Fit the model
nn.fit(X_train_scaled,y_train, 
                    epochs=20,
                    batch_size=100,
                    shuffle=True)

Epoch 1/20


2022-03-15 21:09:21.714059: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-15 21:09:21.897403: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


127/127 [==============================] - 1s 6ms/step - loss: 0.0183 - accuracy: 0.0078
Epoch 2/20
127/127 [==============================] - 1s 6ms/step - loss: 0.0173 - accuracy: 0.0079
Epoch 3/20
127/127 [==============================] - 1s 6ms/step - loss: 0.0142 - accuracy: 0.0079
Epoch 4/20
127/127 [==============================] - 1s 6ms/step - loss: 0.0142 - accuracy: 0.0079
Epoch 5/20
127/127 [==============================] - 1s 6ms/step - loss: 0.0172 - accuracy: 0.0079
Epoch 6/20
127/127 [==============================] - 1s 6ms/step - loss: 0.0138 - accuracy: 0.0079
Epoch 7/20
127/127 [==============================] - 1s 6ms/step - loss: 0.0142 - accuracy: 0.0079
Epoch 8/20
127/127 [==============================] - 1s 6ms/step - loss: 0.0136 - accuracy: 0.0079
Epoch 9/20
127/127 [==============================] - 1s 6ms/step - loss: 0.0136 - accuracy: 0.0079
Epoch 10/20
127/127 [==============================] - 1s 6ms/step - loss: 0.0136 - accuracy: 0.0079
Epoch 11/2

In [11]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2022-03-15 21:09:36.740196: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


132/132 - 0s - loss: 0.0094 - accuracy: 0.0057 - 426ms/epoch - 3ms/step
Loss: 0.009402480907738209, Accuracy: 0.0057007125578820705


## Save the Model and its Weights

In [12]:
# Save model as JSON
nn_json = nn.to_json()

file_path = ("../Resources/model.json")
with open(file_path, "w") as json_file:
    json_file.write(nn_json)

In [13]:
# Save weights
file_path = "../Resources/model.h5"
nn.save_weights(file_path)

## Read in the Model with Weights

In [14]:
#Imports
from tensorflow.keras.models import model_from_json

# load json and create model
file_path = ("../Resources/model.json")
with open("../Resources/model.json", "r") as json_file:
    model_json = json_file.read()
loaded_model = model_from_json(model_json)

# load weights into new model
file_path = "../Resources/model.h5"
loaded_model.load_weights(file_path)

In [15]:
# The `layers` attribute stores the various layers in the model as a list
loaded_model.layers

## Read in New Data and Work with Previously Saved Model

In [16]:
# Load in some new data
X = pd.read_csv('../Resources/meet_or_beat_US_new_data.csv')
X.head()

,EPS,forecasted_eps,noOfEsts,before_total_returns
0,3.83,3.72,11,-0.088042
1,-2.14,-2.12,1,-0.108000
2,0.58,0.41,10,-0.218316
3,0.11,0.05,2,0.044483
4,-0.01,0.01,1,-0.238274


In [17]:
new_predictions = loaded_model.predict(X)
new_predictions

2022-03-15 21:09:37.255695: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


array([[0.02625941],
       [0.03529102],
       [0.0274414 ],
       ...,
       [0.0176152 ],
       [0.02758233],
       [0.01421257]], dtype=float32)

In [18]:
new_data = pd.read_csv('../Resources/meet_or_beat_AU.csv')
new_data.head()

,EPS,forecasted_eps,noOfEsts,after_total_returns,before_total_returns
0,0.02,0.11,1,-0.065574,0.026369
1,0.07,0.01,1,-0.079787,0.233333
2,0.25,0.19,2,0.057402,0.001473
3,0.08,0.03,1,0.008572,0.217857
4,-0.18,-0.14,1,0.080547,-0.002414


In [19]:
# Split into training and testing windows
from sklearn.model_selection import train_test_split

y_var = 'after_total_returns'
x_vars = list(new_data.columns)
x_vars.remove(y_var)

X_train, X_test, y_train, y_test = train_test_split(new_data[x_vars], new_data[y_var], random_state=1)

# Create the StandardScaler instance
X_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Scale both the training and testing data from the features dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Freeze Layers

In [20]:
# Freeze the existing layers of the loaded model
for layer in loaded_model.layers[0:-1]:
    layer.trainable = False

In [21]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                320       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 24)                792       
                                                                 
 dropout_1 (Dropout)         (None, 24)                0         
                                                                 
 dense_3 (Dense)             (None, 12)                300       
                                                                 
 dense_4 (Dense)             (None, 1)                 1

## Add New Layers to Existing Model

In [22]:
# Create a new DNN to hold the old one
transfer_model = Sequential()
# Go through each layer, skipping the last layer
for layer in loaded_model.layers[:-1]: 
    transfer_model.add(layer)

In [23]:
transfer_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                320       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 24)                792       
                                                                 
 dropout_1 (Dropout)         (None, 24)                0         
                                                                 
 dense_3 (Dense)             (None, 12)                300       
                                                                 
Total params: 3,492
Trainable params: 0
Non-trainable 

In [24]:
# Add an additional layer
transfer_model.add(Dense(10, activation="relu"))
# Add the final output layer
transfer_model.add(Dense(1))

In [25]:
transfer_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                320       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 24)                792       
                                                                 
 dropout_1 (Dropout)         (None, 24)                0         
                                                                 
 dense_3 (Dense)             (None, 12)                300       
                                                                 
 dense_5 (Dense)             (None, 10)               

## Fit the Revised Model

In [26]:
# Compile the Sequential model
transfer_model.compile(loss="mean_absolute_error", optimizer="adam", metrics=["accuracy"])

In [27]:
# Fit the model
transfer_model.fit(X_train_scaled,y_train, 
                    epochs=20,
                    batch_size=100,
                    shuffle=True)

Epoch 1/20
22/41 [===============>..............] - ETA: 0s - loss: 0.0640 - accuracy: 0.0027  

2022-03-15 21:09:37.571670: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


41/41 [==============================] - 0s 5ms/step - loss: 0.0637 - accuracy: 0.0049
Epoch 2/20
41/41 [==============================] - 0s 5ms/step - loss: 0.0629 - accuracy: 0.0049
Epoch 3/20
41/41 [==============================] - 0s 5ms/step - loss: 0.0624 - accuracy: 0.0049
Epoch 4/20
41/41 [==============================] - 0s 5ms/step - loss: 0.0623 - accuracy: 0.0049
Epoch 5/20
41/41 [==============================] - 0s 5ms/step - loss: 0.0623 - accuracy: 0.0049
Epoch 6/20
41/41 [==============================] - 0s 5ms/step - loss: 0.0628 - accuracy: 0.0049
Epoch 7/20
41/41 [==============================] - 0s 5ms/step - loss: 0.0628 - accuracy: 0.0049
Epoch 8/20
41/41 [==============================] - 0s 5ms/step - loss: 0.0628 - accuracy: 0.0049
Epoch 9/20
41/41 [==============================] - 0s 5ms/step - loss: 0.0623 - accuracy: 0.0049
Epoch 10/20
41/41 [==============================] - 0s 5ms/step - loss: 0.0622 - accuracy: 0.0049
Epoch 11/20
41/41 [===========

In [28]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

43/43 - 0s - loss: 0.0086 - accuracy: 0.0073 - 185ms/epoch - 4ms/step
Loss: 0.008577074855566025, Accuracy: 0.007342143915593624


2022-03-15 21:09:41.721737: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
